# Day 4

## GENERIC SETUP

In [5]:
# General imports
import pytest
import ipytest
import time
import functools

# Setup ipytest
ipytest.autoconfig()

# Setup nb_black
%load_ext nb_black

# Decorator to time solutions
def timer(func):
    """
    Wrapper function.
    Print the runtime of the decorated function.
    """

    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()  # 1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()  # 2
        run_time = end_time - start_time  # 3
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value

    return wrapper_timer

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

## SOLUTION SETUP

In [11]:
# Solution-specific imports
import re

# What day do we solve? Used to identify the input datafile, integer value
DAY = 4

<IPython.core.display.Javascript object>

#### I/O functions

In [7]:
def get_input():
    with open(f"../data/{DAY}.txt", "r") as f:
        return split_input(f.read())


def split_input(input_raw):
    # Split on double newlines
    return list(map(str.strip, input_raw.split("\n\n")))

<IPython.core.display.Javascript object>

#### Pytest input data

In [84]:
# Sample input
@pytest.fixture
def dummy_input_A():
    return """\
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in"""


@pytest.fixture
def invalid_passports_B():
    return """\
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007"""


@pytest.fixture
def valid_passports_B():
    return """\
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719"""

<IPython.core.display.Javascript object>

## Solution A

In [69]:
def create_passport(raw_passport):
    """
    Creates a proper passport dict from
    a list of raw k/v-pair strings.
    """
    passport = {
        str(raw_pair.split(":")[0]): str(raw_pair.split(":")[1])
        for raw_pair in raw_passport
    }
    return passport


def validate_passport_A(passport):
    """
    If at least one of the required fields is not
    a key in the dictionary, return False, else True
    """
    req_fields = [
        "byr",
        "iyr",
        "eyr",
        "hgt",
        "hcl",
        "ecl",
        "pid",
    ]

    return all(field in passport for field in req_fields)


@timer
def solve_A(input):
    """
    IMPLEMENT ME
    """
    # 1. Split each input into k/v pairs on whitespace or newline.
    # Define once, use often for efficiency.
    split_pattern = re.compile("[\s\n]+")
    # Apply to each raw passport in input
    input_split = map(split_pattern.split, input)

    # 2. Create passports
    passports = map(create_passport, input_split)

    # 3. Check each passport for required fields
    req_fields = [
        "byr",
        "iyr",
        "eyr",
        "hgt",
        "hcl",
        "ecl",
        "pid",
    ]
    valid_checks = map(validate_passport_A, passports)

    # Return the number of valid checks
    return sum(valid_checks)

<IPython.core.display.Javascript object>

#### Tests

In [70]:
%%run_pytest[clean] -qq

def test_A(dummy_input_A):
    assert solve_A(split_input(dummy_input_A)) == 2

<IPython.core.display.Javascript object>

.                                                                                                                                                                                                                                      [100%]


<IPython.core.display.Javascript object>

#### OUTPUT

In [71]:
solve_A(get_input())

Finished 'solve_A' in 0.0027 secs


219

<IPython.core.display.Javascript object>

## Solution B

In [103]:
def validate_numeric(value, criteria):
    """
    Validates [value] to:
    [min_value]<=[value]<=[max_value]
    and to have a positive regex match to [match_pattern]
    """
    pattern, min_val, max_val = criteria

    if pattern.match(value):
        return min_val <= int(value) <= max_val
    else:
        return False


def validate_height(value, criteria):
    """
    Validates [value] to:
    [min_value]<=[value]<=[max_value]
    and to have a positive regex match to [match_pattern]
    with varying min and max values for cm and inches
    """
    pattern, min_cm, max_cm, min_in, max_in = criteria

    numeric_value = int(re.sub("\D", "", value))
    if "cm" in value:
        if pattern.match(value):
            return min_cm <= numeric_value <= max_cm
    elif "in" in value:
        if pattern.match(value):
            return min_in <= numeric_value <= max_in
    else:
        # Unknown measurement or faulty formatting
        return False


def validate_pattern(value, pattern):
    """
    Validates [value] to:
    Have a positive regex match to [match_pattern]
    """
    if pattern.match(value):
        return True
    else:
        return False


def validate_passport_B(passport, req_fields, val_funcs, val_criteria):
    """
    If at least one of the required fields is not
    a key in the dictionary, return False.

    If at least one of the fields fails validation,
    return False.
    """
    # 1. Check required fields
    all_fields_present = all(field in passport for field in req_fields)
    if not all_fields_present:
        return False

    # 2. If all fields present, check their value for validity
    checks = [
        val_funcs[k](v, val_criteria[k]) for k, v in passport.items() if k in req_fields
    ]

    # 3. If all checks pass, return True, else False
    return all(checks)


@timer
def solve_B(input):
    """
    Transform raw input (split to raw passports) to passport dicts,
    run the validity check on each passport,
    and count the number of fully valid passports.
    """
    # 0. Prepare validation functions and criteria once for re-use
    validation_criteria = {
        "byr": [re.compile("\d{4}"), 1920, 2002],
        "iyr": [re.compile("\d{4}"), 2010, 2020],
        "eyr": [re.compile("\d{4}"), 2020, 2030],
        "hgt": [re.compile("\d+(cm|in)"), 150, 193, 59, 76],
        "hcl": re.compile("#[0-9a-f]{6}"),
        "ecl": re.compile("(amb|blu|brn|gry|grn|hzl|oth)"),
        "pid": re.compile("^\d{9}$"),
    }

    validation_functions = {
        "byr": validate_numeric,
        "iyr": validate_numeric,
        "eyr": validate_numeric,
        "hgt": validate_height,
        "hcl": validate_pattern,
        "ecl": validate_pattern,
        "pid": validate_pattern,
    }

    req_fields = [
        "byr",
        "iyr",
        "eyr",
        "hgt",
        "hcl",
        "ecl",
        "pid",
    ]

    # 1. Split each input into k/v pairs on whitespace or newline.
    # Define once, use often for efficiency.
    split_pattern = re.compile("[\s\n]+")
    # Apply to each raw passport in input
    input_split = map(split_pattern.split, input)

    # 2. Create passports
    passports = map(create_passport, input_split)

    # 3. Check each passport for validity using the criteria dicts
    valid_checks = [
        validate_passport_B(
            passport, req_fields, validation_functions, validation_criteria
        )
        for passport in passports
    ]

    # Return the number of valid checks
    return sum(valid_checks)

<IPython.core.display.Javascript object>

#### Tests

In [104]:
%%run_pytest[clean] -qq

@pytest.mark.parametrize("test_value, expected", [
    ("2002", True),
    ("2003", False),
    ("42", False),
])
def test_validate_numeric(test_value, expected):
    criteria = [re.compile("\d{4}"), 1920, 2002]
    assert validate_numeric(test_value, criteria) == expected


@pytest.mark.parametrize("test_value, expected", [
    ("60in", True),
    ("190cm", True),
    ("190in", False),
    ("190", False),
])
def test_validate_height(test_value, expected):
    criteria = [re.compile("\d+(cm|in)"), 150, 193, 59, 76]
    assert validate_height(test_value, criteria) == expected


@pytest.mark.parametrize("test_value, criteria, expected", [
    # Hair colours hcl
    ("#123abz", re.compile("#[0-9a-f]{6}"), False),
    ("#123abc", re.compile("#[0-9a-f]{6}"), True),
    ("123abc", re.compile("#[0-9a-f]{6}"), False),
    # Eye colour ecl
    ("brn", re.compile("(amb|blu|brn|gry|grn|hzl|oth)"), True),
    ("wat", re.compile("(amb|blu|brn|gry|grn|hzl|oth)"), False),
    # Passport ID pid
    ("000000001", re.compile("^\d{9}$"), True),
    ("0123456789", re.compile("^\d{9}$"), False),
])
def test_validate_pattern(test_value, criteria, expected):
    assert validate_pattern(test_value, criteria) == expected


def test_B(dummy_input):
    assert solve_B(split_input(dummy_input)) == 2


def test_B_all_invalid(invalid_passports_B):
    assert solve_B(split_input(invalid_passports_B)) == 0
    

def test_B_all_valid(valid_passports_B):
    assert solve_B(split_input(valid_passports_B)) == 4

<IPython.core.display.Javascript object>

.................                                                                                                                                                                                                                      [100%]


<IPython.core.display.Javascript object>

#### OUTPUT

In [105]:
solve_B(get_input())

Finished 'solve_B' in 0.0079 secs


127

<IPython.core.display.Javascript object>